# Object Detection

## Import everything

In [ ]:
import os
import sys

import cv2
import numpy as np
import yaml

from sc_sdk.entities.datasets import Subset
from sc_sdk.entities.dataset_storage import NullDatasetStorage
from sc_sdk.entities.id import ID
from sc_sdk.entities.inference_parameters import InferenceParameters
from sc_sdk.entities.model import Model, ModelStatus, NullModel
from sc_sdk.entities.model_storage import NullModelStorage
from sc_sdk.entities.optimized_model import ModelOptimizationType, ModelPrecision, OptimizedModel, TargetDevice
from sc_sdk.entities.project import NullProject
from sc_sdk.entities.resultset import ResultSet
from sc_sdk.entities.task_environment import TaskEnvironment
from sc_sdk.usecases.tasks.interfaces.export_interface import ExportType

from ote_cli.datasets import get_dataset_class
from ote_cli.registry import Registry
from ote_cli.utils.config import apply_template_configurable_parameters
from ote_cli.utils.labels import generate_label_schema
from ote_cli.utils.loading import load_config
from ote_cli.utils.importing import get_impl_class

## Register templates

In [ ]:
templates_dir = '../../model_templates'
registry = Registry(templates_dir)
registry = registry.filter(framework=sys.executable.split(os.sep)[-4])

print(registry)

## Load model template

In [ ]:
template = registry.get('Custom_Object_Detection_256x256')
config = load_config(template['path'])
config['hyper_parameters']['params'].setdefault('algo_backend', {})['template'] = os.path.abspath(template['path'])

## Get training task and dataset instantiated

In [ ]:
Task = get_impl_class(config['task']['base'])
ConfigurableParameters = get_impl_class(config['hyper_parameters']['impl'])
Dataset = get_dataset_class(config['domain'])

dataset_paths = {
    'train_ann_file' : '../../data/airport/annotation_faces_train.json',
    'train_data_root' : '../../data/airport/',
    'val_ann_file' : '../../data/airport/annotation_faces_train.json',
    'val_data_root' : '../../data/airport/',
}

dataset = Dataset(**dataset_paths, dataset_storage=NullDatasetStorage())

params = ConfigurableParameters(workspace_id=ID(), model_storage_id=ID())
apply_template_configurable_parameters(params, config)

labels_schema = generate_label_schema(dataset.get_labels(), config['domain'])
dataset.set_project_labels(labels_schema.get_labels(False))

## Have a look at existing parameters

In [ ]:
for p in params.learning_parameters.parameters:
    print(f'{p}: {getattr(params.learning_parameters, p)}')

## Tweak parameters

In [ ]:
params.learning_parameters.batch_size = 16
params.learning_parameters.num_iters = 100

for p in params.learning_parameters.parameters:
    print(f'{p}: {getattr(params.learning_parameters, p)}')

## Create Task 

In [ ]:
environment = TaskEnvironment(model=NullModel(), hyper_parameters=params, label_schema=labels_schema)
task = Task(task_environment=environment)

## Run training

In [ ]:
output_model = Model(
    NullProject(),
    NullModelStorage(),
    dataset,
    environment.get_model_configuration(),
    model_status=ModelStatus.NOT_READY)

task.train(dataset, output_model)

## Evaluate quality metric

In [ ]:
validation_dataset = dataset.get_subset(Subset.VALIDATION)
predicted_validation_dataset = task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))

resultset = ResultSet(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
performance = task.evaluate(resultset)
print(performance)


## Export model to OpenVINO format

In [ ]:
exported_model = OptimizedModel(
    NullProject(),
    NullModelStorage(),
    dataset,
    environment.get_model_configuration(),
    ModelOptimizationType.MO,
    precision=[ModelPrecision.FP16],
    optimization_methods=[],
    optimization_level={},
    target_device=TargetDevice.UNSPECIFIED,
    performance_improvement={},
    model_size_reduction=1.,
    model_status=ModelStatus.NOT_READY)
task.export(ExportType.OPENVINO, exported_model)

## Evaluate the exported model

In [ ]:
environment.model = exported_model
ov_task = get_impl_class(config['task']['openvino'])(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSet(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
performance = ov_task.evaluate(resultset)
print(performance)

## Draw bounding boxes

In [ ]:
import IPython
import PIL

for predictions, item in zip(predicted_validation_dataset, validation_dataset.with_empty_annotations()):
    image = item.numpy.astype(np.uint8)
    for box in predictions.annotation_scene.shapes:
        x1 = int(box.x1 * image.shape[1])
        x2 = int(box.x2 * image.shape[1])
        y1 = int(box.y1 * image.shape[0])
        y2 = int(box.y2 * image.shape[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    IPython.display.display(PIL.Image.fromarray(image))